In [1]:
# Import modules
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plot
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.externals import joblib
from sklearn.metrics import r2_score
print('All modules successfully imported.')

All modules successfully imported.


In [2]:
# Define user input variables
print('Enter root directory:')
root_folder = 'E:\VegetationEcology\Data_Harmonization\GoogleCloud'
print('Enter name of output folder:')
output_folder = 'E:\VegetationEcology\Data_Harmonization\GoogleCloud\output_SalixPulchra'
print('Enter name of predict folder:')
input_data_name = 'salix_pulchra.csv'
print('Enter name of output report file:')

Enter root directory:
Enter name of output folder:
Enter name of predict folder:
Enter name of output report file:


In [41]:
# Define variable sets
predictor_variables = ['compoundTopographic', 'dateFreeze_2000s', 'dateThaw_2000s', 'elevation', 'floodplainsDist', 'growingSeason_2000s', 'heatLoad', 'integratedMoisture', 'precipAnnual_2000s', 'roughness', 'siteExposure', 'slope', 'streamLargeDist', 'streamSmallDist', 'summerWarmth_2000s', 'surfaceArea', 'surfaceRelief', 'aspect', 'F05May_1_ultraBlue', 'F05May_2_blue', 'F05May_3_green', 'F05May_4_red', 'F05May_5_nearInfrared', 'F05May_6_shortInfrared1', 'F05May_7_shortInfrared2', 'F05May_evi2', 'F05May_nbr', 'F05May_ndmi', 'F05May_ndsi', 'F05May_ndvi', 'F05May_ndwi', 'F06June_1_ultraBlue', 'F06June_2_blue', 'F06June_3_green', 'F06June_4_red', 'F06June_5_nearInfrared', 'F06June_6_shortInfrared1', 'F06June_7_shortInfrared2', 'F06June_evi2', 'F06June_nbr', 'F06June_ndmi', 'F06June_ndsi', 'F06June_ndvi', 'F06June_ndwi', 'F07July_1_ultraBlue', 'F07July_2_blue', 'F07July_3_green', 'F07July_4_red', 'F07July_5_nearInfrared', 'F07July_6_shortInfrared1', 'F07July_7_shortInfrared2', 'F07July_evi2', 'F07July_nbr', 'F07July_ndmi', 'F07July_ndsi', 'F07July_ndvi', 'F07July_ndwi', 'F08August_1_ultraBlue', 'F08August_2_blue', 'F08August_3_green', 'F08August_4_red', 'F08August_5_nearInfrared', 'F08August_6_shortInfrared1', 'F08August_7_shortInfrared2', 'F08August_evi2', 'F08August_nbr', 'F08August_ndmi', 'F08August_ndsi', 'F08August_ndvi', 'F08August_ndwi', 'F09September_1_ultraBlue', 'F09September_2_blue', 'F09September_3_green', 'F09September_4_red', 'F09September_5_nearInfrared', 'F09September_6_shortInfrared1', 'F09September_7_shortInfrared2', 'F09September_evi2', 'F09September_nbr', 'F09September_ndmi', 'F09September_ndsi', 'F09September_ndvi', 'F09September_ndwi']
predictor_subset = ['compoundTopographic', 'dateFreeze_2000s', 'dateThaw_2000s', 'elevation', 'floodplainsDist', 'growingSeason_2000s', 'heatLoad', 'integratedMoisture', 'precipAnnual_2000s', 'roughness', 'siteExposure', 'slope', 'streamLargeDist', 'streamSmallDist', 'summerWarmth_2000s', 'surfaceArea', 'surfaceRelief', 'aspect', 'F05May_2_blue', 'F05May_evi2', 'F05May_nbr', 'F05May_ndmi', 'F05May_ndsi', 'F05May_ndvi', 'F05May_ndwi', 'F06June_2_blue', 'F06June_evi2', 'F06June_nbr', 'F06June_ndmi', 'F06June_ndsi', 'F06June_ndvi', 'F06June_ndwi', 'F07July_2_blue', 'F07July_evi2', 'F07July_nbr', 'F07July_ndmi', 'F07July_ndsi', 'F07July_ndvi', 'F07July_ndwi', 'F08August_2_blue', 'F08August_evi2', 'F08August_nbr', 'F08August_ndmi', 'F08August_ndsi', 'F08August_ndvi', 'F08August_ndwi', 'F09September_2_blue', 'F09September_evi2', 'F09September_nbr', 'F09September_ndmi', 'F09September_ndsi', 'F09September_ndvi', 'F09September_ndwi']
zero_variable = ['zero']
ten_variable = ['ten']
twentyfive_variable = ['twentyfive']
cover = ['cover']
strata = ['strata']
retain_variables = ['project', 'siteID', 'siteCode', 'methodSurvey', 'methodCover']
coordinates = ['POINT_X', 'POINT_Y']
all_variables = predictor_variables + zero_variable + ten_variable + twentyfive_variable + cover + strata + retain_variables + coordinates
print('Variable sets loaded.')

Variable sets loaded.


In [4]:
# Define a function to plot variable importances
def plotVariableImportances(inModel, x_train, outVariableFile):
    # Get numerical feature importances
    importances = list(inModel.feature_importances_)
    # List of tuples with variable and importance
    feature_list = list(x_train.columns)
    feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
    # Sort the feature importances by most important first
    feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    # Initialize the plot and set figure size
    variable_figure = plot.figure()
    plot.style.use('fivethirtyeight')
    fig_size = plot.rcParams["figure.figsize"]
    fig_size[0] = 12
    fig_size[1] = 9
    plot.rcParams["figure.figsize"] = fig_size
    # Create list of x locations for plotting
    x_values = list(range(len(importances)))
    # Make a bar chart of the variable importances
    plot.bar(x_values, importances, orientation = 'vertical')
    # Tick labels for x axis
    plot.xticks(x_values, feature_list, rotation='vertical')
    # Axis labels and title
    plot.ylabel('Importance'); plot.xlabel('Variable'); plot.title('Variable Importances');
    # Export
    variable_figure.savefig(outVariableFile, bbox_inches="tight", dpi=300)
    # Clear plot workspace
    plot.clf()
    plot.close()
    
print('Function "plotVariableImportances" loaded.')

Function "plotVariableImportances" loaded.


In [5]:
# Define a function to calculate performance metrics based on a specified threshold value
def thresholdMetrics(inIndex, inProbability, inValue, y_test):
    outThresholded = np.zeros(inIndex.shape)
    outThresholded[inIndex > inValue] = 1
    confusion_test = confusion_matrix(y_test, outThresholded)
    true_negative = confusion_test[0,0]
    false_negative = confusion_test[1,0]
    true_positive = confusion_test[1,1]
    false_positive = confusion_test[0,1]
    outSensitivity = true_positive / (true_positive + false_negative)
    outSpecificity = true_negative / (true_negative + false_positive)
    outAUC = roc_auc_score(y_test, inProbability)
    outAccuracy = (true_negative + true_positive) / (true_negative + false_positive + false_negative + true_positive)
    return (outThresholded, outSensitivity, outSpecificity, outAUC, outAccuracy)

print('Function "thresholdMetrics" loaded.')

Function "thresholdMetrics" loaded.


In [55]:
# Define a function to fit a classifier using training data and determine a best classification threshold using the test data
def trainTestModel(X_train, y_train, X_test, y_test, testData, variable):
    # Fit a random forest classifier to the training dataset
    rf_classify = RandomForestClassifier(n_estimators = 5000, criterion='entropy', max_features='sqrt', bootstrap = True, oob_score = True, n_jobs=1, class_weight = 'balanced')
    rf_classify.fit(X_train, y_train)
    # Use the random forest classifier to predict probabilities for the test dataset
    test_prediction = rf_classify.predict_proba(X_test)
    # Convert the positive class probabilities to a list of probabilities
    test_probability = [p[1] for p in test_prediction]
    # Convert the postitive class probabilities to an index between 0 and 1000
    test_index = [int((p[1] * 1000) + 0.5) for p in test_prediction]
    # Iterate through numbers between 0 and 1000 to output a list of sensitivity and specificity values per threshold number
    i = 1
    test_index = np.asarray(test_index)
    sensitivity_list = []
    specificity_list = []
    while i < 1001:
        test_thresholded, sensitivity_test, specificity_test, auc_test, accuracy_test = thresholdMetrics(test_index, test_probability, i, y_test)
        sensitivity_list.append(sensitivity_test)
        specificity_list.append(specificity_test)
        i = i + 1
    # Calculate a list of absolute value of difference between sensitivity and specificity and find the optimal threshold
    difference_list = [a - b for a, b in zip(sensitivity_list, specificity_list)]
    value, threshold = min((value, threshold) for (threshold, value) in enumerate(difference_list) if value >= 0)
    # Calculate the prediction index to a binary 0 or 1 output using the optimal threshold
    test_thresholded, sensitivity_test, specificity_test, auc_test, accuracy_test = thresholdMetrics(test_index, test_probability, threshold, y_test)
    # Concatenate thresholded predictions to test data frame
    testData = pd.concat([testData, pd.DataFrame(test_thresholded)], axis=1)
    testData = testData.rename(index=int, columns={0: variable})
    return [threshold, sensitivity_test, specificity_test, auc_test, accuracy_test, rf_classify, testData]

print('Function "trainTestModel" loaded.')

Function "trainTestModel" loaded.


In [7]:
# Set initial plot sizefig_size = plot.rcParams["figure.figsize"]
fig_size = plot.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plot.rcParams["figure.figsize"] = fig_size
print('Plot size parameters configured.')

Plot size parameters configured.


In [8]:
# Import input data from csv file
input_file = os.path.join(os.path.join(root_folder, "speciesData"), input_data_name)
input_df = pd.read_csv(input_file)
# Convert numerical data to integers
input_df[predictor_variables + zero_variable + ten_variable + twentyfive_variable + cover + strata] = input_df[predictor_variables + zero_variable + ten_variable + twentyfive_variable + cover + strata].astype(float)
print(input_df)

      cover        project  siteID          siteCode          methodSurvey  \
0       0.0        NSSI LC    1975      NSSI11_04_01       Visual Estimate   
1       0.0        NSSI LC    1976      NSSI11_04_02       Visual Estimate   
2       0.0        NSSI LC    2034      NSSI11_04_04       Visual Estimate   
3       0.0        NSSI LC    1987      NSSI11_04_03       Visual Estimate   
4       0.0      AIM NPR-A     180           TMCW-64  Line-Point Intercept   
5       1.0      AIM NPR-A     182           TMCW-68  Line-Point Intercept   
6       0.0      AIM NPR-A     178           TMCW-61  Line-Point Intercept   
7       0.0      AIM NPR-A      64          CPBWM-69  Line-Point Intercept   
8       0.0      AIM NPR-A     181           TMCW-67  Line-Point Intercept   
9       0.0      AIM NPR-A     179           TMCW-63  Line-Point Intercept   
10      0.0      AIM NPR-A      65          CPBWM-70  Line-Point Intercept   
11      5.0      AIM NPR-A      82          CPHCP-59  Line-Point

In [9]:
# Create a plots folder if it does not exist
plots_folder = os.path.join(output_folder, "plots")
if not os.path.exists(plots_folder):
    os.makedirs(plots_folder)
    print('Plots folder created.')
else:
    print('Plots folder already exists.')

Plots folder already exists.


In [44]:
# Create train and test splits
X = input_df[all_variables]
y = np.array(input_df[zero_variable]).ravel()
strata = np.array(input_df['strata']).ravel()
All_train, All_test, y_train, y_test = train_test_split(X, y, test_size=0.3, train_size = 0.7, random_state = None, shuffle = True, stratify = strata)
testData = All_test.reset_index()
print(testData)

     index  compoundTopographic  dateFreeze_2000s  dateThaw_2000s  elevation  \
0      415               1093.0             268.0           149.0       10.0   
1     1856               1121.0             263.0           128.0     1088.0   
2     2073                746.0             275.0           126.0      327.0   
3      555               1100.0             267.0           147.0       40.0   
4     2125               1061.0             279.0           123.0        2.0   
5     1651               1549.0             269.0           123.0      603.0   
6      201               1461.0             268.0           150.0       10.0   
7     1676               1100.0             276.0           129.0       16.0   
8     1968                839.0             274.0           115.0      246.0   
9     1597                983.0             269.0           129.0      475.0   
10    1158                580.0             261.0           136.0      970.0   
11    2645                935.0         

In [40]:
# Classify and predict the zero class
X_train = All_train[predictor_variables]
y_train = All_train['zero']
X_test = All_test[predictor_variables]
y_test = All_test['zero']
threshold_0, sensitivity_0, specificity_0, auc_0, accuracy_0, classifier_0, testData = trainTestModel(X_train, y_train, X_test, y_test, testData, 'predict_0')
print(classifier_0)
print(auc_0)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=None, max_features=3,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=5000, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)
0.8100265550271647


In [45]:
# Subset the train data
Subset_train = All_train[All_train['strata'] >= 1]
Subset_test = All_test[All_test['strata'] >= 1]
print(Subset_train)

      compoundTopographic  dateFreeze_2000s  dateThaw_2000s  elevation  \
1622                896.0             268.0           128.0      547.0   
1153               1282.0             264.0           137.0      459.0   
792                1512.0             267.0           143.0      171.0   
795                 972.0             266.0           142.0      337.0   
120                1152.0             269.0           151.0        6.0   
28                 1216.0             269.0           151.0       12.0   
1805               1076.0             276.0           122.0       99.0   
177                 948.0             268.0           149.0       18.0   
1452                853.0             265.0           126.0      860.0   
2579                837.0             274.0           114.0      605.0   
1386                996.0             265.0           135.0      630.0   
1800                950.0             278.0           128.0      116.0   
1400                832.0             

In [56]:
# Classify and predict the ten class
X_train = Subset_train[predictor_subset]
y_train = Subset_train['ten']
X_test = All_test[predictor_subset]
y_test = All_test['ten']
threshold_10, sensitivity_10, specificity_10, auc_10, accuracy_10, classifier_10, testData = trainTestModel(X_train, y_train, X_test, y_test, testData, 'cover_10')
print(classifier_10)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=None, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=5000, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)


In [57]:
print(auc_10)

0.7741821020509545


In [16]:
# Classify and predict the twentyfive class
X_train = Subset_train[predictor_variables]
y_train = Subset_train['twentyfive']
X_test = All_test[predictor_variables]
y_test = All_test['twentyfive']
threshold_25, sensitivity_25, specificity_25, auc_25, accuracy_25, classifier_25, testData = trainTestModel(X_train, y_train, X_test, y_test, testData, 'cover_25')
print(classifier_25)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=5000, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)


In [26]:
print(auc_25)

0.8060047434238895
